In [8]:
# ============================================================================
# MULTI-RANGE BARABÁSI-ALBERT MODEL WITH CHECKPOINTING
# ============================================================================
# Enhanced BA model with 3 connection ranges and crash recovery
# ============================================================================

# === INSTALLATION ===
!pip install networkit pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import networkit as nk
import time
import gc
import pickle
import shutil
from pathlib import Path
from collections import defaultdict
import sys
from scipy import stats
from scipy.stats import t

# === CONFIGURATION ===
print(f"{'=' * 70}")
print(f"MULTI-RANGE BA MODEL CONFIGURATION")
print(f"{'=' * 70}")

# Simulation parameters - 500 total simulations
NUM_BOOSTING_ROUNDS = 20
BRIDGINGS_PER_ROUND = 25
TOTAL_SIMULATIONS = NUM_BOOSTING_ROUNDS * BRIDGINGS_PER_ROUND  # 500

# THREE CONNECTION RANGES
CONNECTION_RANGES = [
    (5, 35, "Range_5-35"),
    (35, 65, "Range_35-65"),
    (65, 95, "Range_65-95"),
]

print(
    f"Simulations per run: {TOTAL_SIMULATIONS} ({NUM_BOOSTING_ROUNDS}×{BRIDGINGS_PER_ROUND})"
)
print(f"Connection Ranges:")
for min_c, max_c, name in CONNECTION_RANGES:
    print(f"  - {name}: {min_c}-{max_c} connections")
print(f"{'=' * 70}\n")

# SEO parameters
NEUTRAL_THRESHOLD = 0.025
PAGERANK_TOLERANCE = 1e-6
PAGERANK_DAMPING = 0.80

# BA graph parameters
TOTAL_NODES_WWW = 500000
EDGES_PER_NEW_NODE = 3

# File paths
BASELINE_PATH = "/content/drive/MyDrive/WebKnoGraph/results/link_graph_edges.csv"

COMPARISON_FOLDERS = [
    (
        "/content/drive/MyDrive/WebKnoGraph/results/expert_led/random_batches/",
        "Random Candidates",
    ),
    (
        "/content/drive/MyDrive/WebKnoGraph/results/expert_led/high_batches/",
        "High Candidates",
    ),
    (
        "/content/drive/MyDrive/WebKnoGraph/results/expert_led/folder_batches/",
        "Folder Candidates",
    ),
    (
        "/content/drive/MyDrive/WebKnoGraph/results/expert_led/mixed_batches/",
        "Mixed Candidates",
    ),
    (
        "/content/drive/MyDrive/WebKnoGraph/results/expert_led/low_batches/",
        "Low Candidates",
    ),
]

# Output directories
OUTPUT_DIR = Path("/content/drive/MyDrive/WebKnoGraph/results/")
MULTI_RANGE_OUTPUT_DIR = OUTPUT_DIR / "multi_range_analysis_ba/"
CHECKPOINT_DIR = MULTI_RANGE_OUTPUT_DIR / "checkpoints_ba/"

MULTI_RANGE_OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
CHECKPOINT_DIR.mkdir(exist_ok=True, parents=True)

print(f"📁 Output: {MULTI_RANGE_OUTPUT_DIR}")
print(f"📁 Checkpoints: {CHECKPOINT_DIR}\n")

# Global cache
_www_graph_cache = None
_www_graph_generated = False


# ============================================
# CHECKPOINT SYSTEM
# ============================================


class CheckpointManager:
    """Save/load checkpoints for crash recovery"""

    def __init__(self, checkpoint_dir=CHECKPOINT_DIR):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(exist_ok=True, parents=True)

    def save_checkpoint(self, range_name, strategy_name, results):
        """Save checkpoint after each strategy completion"""
        checkpoint_id = f"{range_name}_{strategy_name.replace(' ', '_')}"
        checkpoint_path = self.checkpoint_dir / f"{checkpoint_id}.pkl"

        try:
            with open(checkpoint_path, "wb") as f:
                pickle.dump(results, f)
            print(f"  ✓ Checkpoint saved: {checkpoint_id}")
        except Exception as e:
            print(f"  ⚠ Checkpoint save failed: {e}")

    def load_checkpoint(self, range_name, strategy_name):
        """Load checkpoint if exists"""
        checkpoint_id = f"{range_name}_{strategy_name.replace(' ', '_')}"
        checkpoint_path = self.checkpoint_dir / f"{checkpoint_id}.pkl"

        if checkpoint_path.exists():
            try:
                with open(checkpoint_path, "rb") as f:
                    return pickle.load(f)
            except Exception as e:
                print(f"  ⚠ Checkpoint load failed: {e}")
        return None

    def list_checkpoints(self):
        """List all available checkpoints"""
        return [f.name for f in self.checkpoint_dir.glob("*.pkl")]

    def clear_checkpoints(self):
        """Clear all checkpoints"""
        try:
            shutil.rmtree(self.checkpoint_dir)
            self.checkpoint_dir.mkdir(exist_ok=True, parents=True)
            print("  ✓ Checkpoints cleared")
        except Exception as e:
            print(f"  ⚠ Checkpoint clear failed: {e}")


# ============================================
# UTILITY FUNCTIONS
# ============================================


def print_progress(message, style="info"):
    """Print formatted progress messages"""
    timestamp = time.strftime("%H:%M:%S")

    if style == "header":
        print(f"\n{'=' * 70}")
        print(f"[{timestamp}] {message}")
        print(f"{'=' * 70}")
    elif style == "section":
        print(f"\n{'─' * 70}")
        print(f"[{timestamp}] {message}")
        print(f"{'─' * 70}")
    elif style == "success":
        print(f"  ✓ [{timestamp}] {message}")
    elif style == "error":
        print(f"  ✗ [{timestamp}] {message}")
    else:
        print(f"  • [{timestamp}] {message}")

    sys.stdout.flush()


def mount_google_drive():
    """Mount Google Drive in Colab environment"""
    try:
        from google.colab import drive

        drive.mount("/content/drive")
        print_progress("Google Drive mounted successfully!", "success")
        return True
    except:
        print_progress("Not in Colab - skipping drive mount", "info")
        return False


def calculate_confidence_interval(data, confidence=0.95):
    """Calculate confidence interval for mean"""
    n = len(data)
    if n < 2:
        return np.mean(data), 0
    mean = np.mean(data)
    se = stats.sem(data)
    margin = se * t.ppf((1 + confidence) / 2, n - 1)
    return mean, margin


# ============================================
# GRAPH LOADING
# ============================================


def load_graph_from_csv_networkit(file_path):
    """Load directed graph from CSV with FROM/TO columns"""
    print_progress(f"Loading graph from {Path(file_path).name}...", "info")

    try:
        df = pd.read_csv(file_path, usecols=["FROM", "TO"])
        df = df.dropna()
        df["FROM"] = df["FROM"].astype(str)
        df["TO"] = df["TO"].astype(str)
    except Exception as e:
        print_progress(f"Error loading {file_path}: {str(e)}", "error")
        return None, None, None

    from_urls = df["FROM"].values
    to_urls = df["TO"].values

    if len(from_urls) == 0:
        print_progress("No valid edges found", "error")
        return None, None, None

    all_urls = np.unique(np.concatenate([from_urls, to_urls]))
    url_to_idx = {url: i for i, url in enumerate(all_urls)}

    g = nk.Graph(n=len(all_urls), weighted=False, directed=True)
    for src_url, tgt_url in zip(from_urls, to_urls):
        g.addEdge(url_to_idx[src_url], url_to_idx[tgt_url])

    print_progress(f"Loaded: {len(all_urls):,} nodes, {len(df):,} edges", "success")
    return g, all_urls, url_to_idx


def create_www_graph_networkit_OPTIMIZED(n_nodes, m_edges, seed=42):
    """Create BA graph ONCE and cache it"""
    global _www_graph_cache, _www_graph_generated

    cache_key = (n_nodes, m_edges, seed)

    if (
        _www_graph_generated
        and _www_graph_cache is not None
        and _www_graph_cache[0] == cache_key
    ):
        print_progress("Using cached BA graph", "success")
        cached_graph = _www_graph_cache[1]
        new_graph = nk.Graph(
            n=cached_graph.numberOfNodes(), weighted=False, directed=True
        )
        for u, v in cached_graph.iterEdges():
            new_graph.addEdge(u, v)
        return new_graph

    print_progress("GENERATING MASTER BARABÁSI-ALBERT GRAPH", "header")
    print_progress(f"Parameters: {n_nodes:,} nodes, m={m_edges}, seed={seed}", "info")

    start_time = time.time()
    nk.setSeed(seed, False)

    print_progress("Generating graph (this may take 30-60 seconds)...", "info")
    generator = nk.generators.BarabasiAlbertGenerator(
        k=m_edges, nMax=n_nodes, n0=m_edges
    )
    www_graph = generator.generate()

    elapsed = time.time() - start_time
    print_progress(
        f"Generated: {www_graph.numberOfNodes():,} nodes, {www_graph.numberOfEdges():,} edges",
        "success",
    )
    print_progress(f"Generation time: {elapsed:.2f}s", "info")

    print_progress("Caching graph for reuse...", "info")
    cached_graph = nk.Graph(n=www_graph.numberOfNodes(), weighted=False, directed=True)
    for u, v in www_graph.iterEdges():
        cached_graph.addEdge(u, v)

    _www_graph_cache = (cache_key, cached_graph)
    _www_graph_generated = True

    print_progress("Graph cached successfully", "success")
    return www_graph


# ============================================
# PAGERANK CALCULATION
# ============================================


def process_configuration_networkit(
    www_graph, kalicube_edges, kalicube_nodes, min_connections, max_connections
):
    """Merge Kalicube graph with WWW and calculate PageRank"""
    kalicube_offset = www_graph.numberOfNodes()
    n_kalicube = len(kalicube_nodes)
    n_www = www_graph.numberOfNodes()

    merged_graph = nk.Graph(n=n_www, weighted=False, directed=True)
    for u, v in www_graph.iterEdges():
        merged_graph.addEdge(u, v)

    for _ in range(n_kalicube):
        merged_graph.addNode()

    if kalicube_edges:
        for src, tgt in kalicube_edges:
            merged_graph.addEdge(src + kalicube_offset, tgt + kalicube_offset)

    # Variable connections within range
    n_connections = np.random.randint(min_connections, max_connections + 1)
    n_www_sample = min(n_connections, n_www)
    n_kalicube_sample = min(n_connections, n_kalicube)

    www_nodes_sample = np.random.choice(n_www, size=n_www_sample, replace=False)
    kalicube_indices = np.random.choice(
        n_kalicube, size=n_kalicube_sample, replace=False
    )

    for www_node_id, kalicube_idx in zip(www_nodes_sample, kalicube_indices):
        merged_graph.addEdge(www_node_id, kalicube_idx + kalicube_offset)

    pagerank_algo = nk.centrality.PageRank(
        merged_graph, damp=PAGERANK_DAMPING, tol=PAGERANK_TOLERANCE
    )
    pagerank_algo.run()
    pagerank_scores = pagerank_algo.scores()

    pagerank_dict = {}
    for i, url in enumerate(kalicube_nodes):
        pagerank_dict[url] = pagerank_scores[i + kalicube_offset]

    return pagerank_dict


def classify_delta(delta_pct):
    """Classify delta as positive, negative, or neutral"""
    if delta_pct > NEUTRAL_THRESHOLD:
        return "positive"
    elif delta_pct < -NEUTRAL_THRESHOLD:
        return "negative"
    else:
        return "neutral"


# ============================================
# SIMULATION EXECUTION
# ============================================


def run_boosting_round(
    round_id,
    www_graph,
    old_edges,
    new_edges,
    old_nodes,
    new_nodes,
    page_deltas_tracker,
    cumulative_deltas,
    min_conn,
    max_conn,
):
    """Run one boosting round with detailed progress"""
    delta_pcts_all = []

    round_start = time.time()

    print(f"\n    ┌─ Round {round_id + 1}/{NUM_BOOSTING_ROUNDS} Starting...")

    for bridging_id in range(BRIDGINGS_PER_ROUND):
        sim_id = round_id * BRIDGINGS_PER_ROUND + bridging_id
        sim_seed = 42 + sim_id

        np.random.seed(sim_seed)
        random.seed(sim_seed)

        sim_start = time.time()
        print(
            f"    │  Sim {bridging_id + 1}/{BRIDGINGS_PER_ROUND} (total: {sim_id + 1}/{TOTAL_SIMULATIONS})...",
            end=" ",
        )
        sys.stdout.flush()

        pagerank_old = process_configuration_networkit(
            www_graph, old_edges, old_nodes, min_conn, max_conn
        )
        pagerank_new = process_configuration_networkit(
            www_graph, new_edges, new_nodes, min_conn, max_conn
        )

        common_urls = set(pagerank_old.keys()) & set(pagerank_new.keys())
        if not common_urls:
            print("⚠ no common URLs")
            continue

        for url in common_urls:
            old_val = pagerank_old[url]
            new_val = pagerank_new[url]
            delta = new_val - old_val
            delta_pct = (delta / max(old_val, 1e-10)) * 100
            delta_pcts_all.append(delta_pct)
            page_deltas_tracker[url].append(delta_pct)

        sim_elapsed = time.time() - sim_start

        all_current = cumulative_deltas + delta_pcts_all
        current_mean = np.mean(all_current) if len(all_current) > 0 else 0

        print(f"✓ {sim_elapsed:.1f}s (mean: {current_mean:+.3f}%)")
        sys.stdout.flush()

    round_elapsed = time.time() - round_start
    round_mean = np.mean(delta_pcts_all) if delta_pcts_all else 0
    print(
        f"    └─ Round {round_id + 1} complete: {round_elapsed:.1f}s | Round mean: {round_mean:+.3f}%"
    )

    return delta_pcts_all, page_deltas_tracker, round_elapsed


def run_boosted_comparison(
    baseline_data, comparison_file, www_graph, min_conn, max_conn, range_name
):
    """Run complete boosted comparison with specific connection range"""
    comparison_name = comparison_file.stem

    print_progress(f"PROCESSING: {comparison_name} @ {range_name}", "section")
    overall_start = time.time()

    g_old, nodes_old, _ = baseline_data
    g_new, nodes_new, _ = load_graph_from_csv_networkit(comparison_file)

    if g_new is None:
        print_progress("Failed to load comparison graph", "error")
        return None

    old_edges = [(u, v) for u, v in g_old.iterEdges()]
    new_edges = [(u, v) for u, v in g_new.iterEdges()]

    print_progress(
        f"Baseline: {len(nodes_old)} nodes, {len(old_edges):,} edges", "info"
    )
    print_progress(
        f"Comparison: {len(nodes_new)} nodes, {len(new_edges):,} edges", "info"
    )
    print_progress(f"Connection range: {min_conn}-{max_conn}", "info")

    del g_new
    gc.collect()

    page_deltas_tracker = defaultdict(list)
    all_deltas = []

    print_progress(
        f"Running {TOTAL_SIMULATIONS} simulations in {NUM_BOOSTING_ROUNDS} rounds...",
        "info",
    )

    for round_id in range(NUM_BOOSTING_ROUNDS):
        round_deltas, page_deltas_tracker, round_elapsed = run_boosting_round(
            round_id,
            www_graph,
            old_edges,
            new_edges,
            nodes_old,
            nodes_new,
            page_deltas_tracker,
            all_deltas,
            min_conn,
            max_conn,
        )
        all_deltas.extend(round_deltas)

    print()

    if not all_deltas:
        print_progress("No results generated", "error")
        return None

    # Classify pages
    print_progress("Classifying page changes...", "info")
    pages_positive = 0
    pages_negative = 0
    pages_neutral = 0

    for url, deltas in page_deltas_tracker.items():
        avg_delta = np.mean(deltas)
        classification = classify_delta(avg_delta)

        if classification == "positive":
            pages_positive += 1
        elif classification == "negative":
            pages_negative += 1
        else:
            pages_neutral += 1

    total_pages = len(page_deltas_tracker)
    all_deltas = np.array(all_deltas)

    final_mean = np.mean(all_deltas)
    final_max = np.max(all_deltas)
    final_min = np.min(all_deltas)
    final_std = np.std(all_deltas)

    total_duration = time.time() - overall_start

    print_progress(
        f"RESULTS: Mean Δ = {final_mean:+.4f}% (min: {final_min:+.3f}%, max: {final_max:+.3f}%)",
        "success",
    )
    print_progress(
        f"Pages: ↑{pages_positive} ↓{pages_negative} →{pages_neutral}", "info"
    )
    print_progress(
        f"Duration: {total_duration:.1f}s ({total_duration / 60:.2f}m)", "info"
    )
    print()

    return {
        "filename": comparison_name,
        "range_name": range_name,
        "min_connections": min_conn,
        "max_connections": max_conn,
        "mean_delta_pct": final_mean,
        "max_delta_pct": final_max,
        "min_delta_pct": final_min,
        "std_delta_pct": final_std,
        "pages_up": pages_positive,
        "pages_down": pages_negative,
        "pages_neutral": pages_neutral,
        "total_pages": total_pages,
        "num_simulations": TOTAL_SIMULATIONS,
        "elapsed_seconds": total_duration,
    }


# ============================================
# STRATEGY PROCESSING
# ============================================


def process_strategy_with_range(
    baseline_data,
    folder_path,
    strategy_name,
    www_graph,
    min_conn,
    max_conn,
    range_name,
    checkpoint_mgr,
):
    """Process one strategy with specific connection range"""
    print(f"\n{'=' * 70}")
    print(f"STRATEGY: {strategy_name} | RANGE: {range_name} ({min_conn}-{max_conn})")
    print(f"{'=' * 70}")

    # Check for checkpoint
    checkpoint = checkpoint_mgr.load_checkpoint(range_name, strategy_name)
    if checkpoint is not None:
        print_progress("✓ Loaded from checkpoint!", "success")
        return checkpoint

    folder_path = Path(folder_path)
    if not folder_path.exists():
        print_progress(f"Folder not found: {folder_path}", "error")
        return None

    csv_files = sorted(list(folder_path.glob("*.csv")))
    if len(csv_files) == 0:
        print_progress(f"No CSV files in {folder_path}", "error")
        return None

    print_progress(f"Found {len(csv_files)} comparison files", "info")

    results = []
    for i, comp_file in enumerate(csv_files, 1):
        print(f"\n  [{i}/{len(csv_files)}] File: {comp_file.name}")
        print(f"  {'-' * 66}")

        result = run_boosted_comparison(
            baseline_data, comp_file, www_graph, min_conn, max_conn, range_name
        )
        if result is not None:
            results.append(result)
        gc.collect()

    if len(results) == 0:
        print_progress("No valid results", "error")
        return None

    # Calculate summary statistics
    all_mean_deltas = [r["mean_delta_pct"] for r in results]
    mean_val, margin = calculate_confidence_interval(all_mean_deltas)

    avg_conn = (min_conn + max_conn) / 2
    efficiency = mean_val / avg_conn if avg_conn > 0 else 0

    result_summary = {
        "strategy_name": strategy_name,
        "range_name": range_name,
        "min_connections": min_conn,
        "max_connections": max_conn,
        "files_analyzed": len(results),
        "total_simulations": len(results) * TOTAL_SIMULATIONS,
        "overall_mean_delta_pct": mean_val,
        "overall_std": np.std(all_mean_deltas),
        "confidence_interval_95": margin,
        "avg_pages_up": np.mean([r["pages_up"] for r in results]),
        "avg_pages_down": np.mean([r["pages_down"] for r in results]),
        "avg_pages_neutral": np.mean([r["pages_neutral"] for r in results]),
        "efficiency": efficiency,
        "file_results": results,
    }

    # Save checkpoint
    checkpoint_mgr.save_checkpoint(range_name, strategy_name, result_summary)

    print_progress(
        f"Strategy complete: Mean Δ = {mean_val:+.4f}% ± {margin:.4f}%", "success"
    )
    print_progress(f"Efficiency: {efficiency:.5f}% per connection", "info")

    gc.collect()
    return result_summary


# ============================================
# VISUALIZATION
# ============================================


def create_multi_range_comparison_plot(all_results, output_dir):
    """Create comprehensive comparison across strategies and ranges"""
    print_progress("Creating multi-range comparison visualization...", "info")

    strategies = sorted(list(set([r["strategy_name"] for r in all_results])))
    ranges = sorted(list(set([r["range_name"] for r in all_results])))

    fig, axes = plt.subplots(2, 2, figsize=(20, 14))
    fig.suptitle(
        "🔬 BARABÁSI-ALBERT: Multi-Range Strategy Analysis",
        fontsize=20,
        fontweight="bold",
    )

    # 1. Mean Delta Heatmap
    ax1 = axes[0, 0]
    heatmap_data = np.zeros((len(strategies), len(ranges)))
    for i, strategy in enumerate(strategies):
        for j, range_name in enumerate(ranges):
            matching = [
                r
                for r in all_results
                if r["strategy_name"] == strategy and r["range_name"] == range_name
            ]
            if matching:
                heatmap_data[i, j] = matching[0]["overall_mean_delta_pct"]

    im = ax1.imshow(heatmap_data, cmap="RdYlGn", aspect="auto")
    ax1.set_xticks(np.arange(len(ranges)))
    ax1.set_yticks(np.arange(len(strategies)))
    ax1.set_xticklabels(ranges, rotation=45, ha="right")
    ax1.set_yticklabels(strategies)
    ax1.set_title("Mean PageRank Delta (%) - Heatmap", fontsize=14, fontweight="bold")

    for i in range(len(strategies)):
        for j in range(len(ranges)):
            text = ax1.text(
                j,
                i,
                f"{heatmap_data[i, j]:.3f}",
                ha="center",
                va="center",
                color="black",
                fontsize=10,
            )

    plt.colorbar(im, ax=ax1, label="Mean Delta %")

    # 2. Line plot with error bars
    ax2 = axes[0, 1]
    colors = ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D", "#6A994E"]
    for i, strategy in enumerate(strategies):
        strategy_data = [r for r in all_results if r["strategy_name"] == strategy]
        strategy_data = sorted(strategy_data, key=lambda x: x["min_connections"])

        x_vals = [r["min_connections"] for r in strategy_data]
        y_vals = [r["overall_mean_delta_pct"] for r in strategy_data]
        yerr = [r["confidence_interval_95"] for r in strategy_data]

        ax2.errorbar(
            x_vals,
            y_vals,
            yerr=yerr,
            marker="o",
            linewidth=2.5,
            label=strategy,
            color=colors[i % len(colors)],
            capsize=5,
        )

    ax2.set_xlabel("Minimum Connections", fontsize=12, fontweight="bold")
    ax2.set_ylabel("Mean PageRank Delta (%)", fontsize=12, fontweight="bold")
    ax2.set_title(
        "Strategy Performance vs Connection Range (95% CI)",
        fontsize=14,
        fontweight="bold",
    )
    ax2.legend(loc="best")
    ax2.grid(True, alpha=0.3)

    # 3. Efficiency plot
    ax3 = axes[1, 0]
    efficiency_data = {}
    for result in all_results:
        strategy = result["strategy_name"]
        if strategy not in efficiency_data:
            efficiency_data[strategy] = []
        efficiency_data[strategy].append(
            (result["min_connections"], result["efficiency"])
        )

    for i, (strategy, data) in enumerate(efficiency_data.items()):
        data = sorted(data, key=lambda x: x[0])
        x_vals = [d[0] for d in data]
        y_vals = [d[1] for d in data]
        ax3.plot(
            x_vals,
            y_vals,
            marker="s",
            linewidth=2.5,
            label=strategy,
            color=colors[i % len(colors)],
        )

    ax3.set_xlabel("Minimum Connections", fontsize=12, fontweight="bold")
    ax3.set_ylabel("Efficiency (Δ% per connection)", fontsize=12, fontweight="bold")
    ax3.set_title("ROI: PageRank Gain per Connection", fontsize=14, fontweight="bold")
    ax3.legend(loc="best")
    ax3.grid(True, alpha=0.3)

    # 4. Summary table
    ax4 = axes[1, 1]
    ax4.axis("tight")
    ax4.axis("off")

    table_data = [["Strategy", "Range", "Mean Δ%", "95% CI", "↑Pages", "Efficiency"]]
    for result in sorted(
        all_results, key=lambda x: (x["range_name"], -x["overall_mean_delta_pct"])
    ):
        table_data.append(
            [
                result["strategy_name"][:12],
                result["range_name"].replace("Range_", ""),
                f"{result['overall_mean_delta_pct']:.3f}",
                f"±{result['confidence_interval_95']:.3f}",
                f"{result['avg_pages_up']:.1f}",
                f"{result['efficiency']:.4f}",
            ]
        )

    table = ax4.table(
        cellText=table_data,
        cellLoc="center",
        loc="center",
        colWidths=[0.22, 0.15, 0.13, 0.13, 0.12, 0.15],
    )
    table.auto_set_font_size(False)
    table.set_fontsize(7)
    table.scale(1, 2)

    for i in range(6):
        table[(0, i)].set_facecolor("#40466e")
        table[(0, i)].set_text_props(weight="bold", color="white")

    plt.tight_layout()

    output_path = output_dir / "multi_range_comparison_ba.png"
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()

    print_progress(f"Saved: {output_path.name}", "success")


def create_detailed_analysis_report(all_results, output_dir):
    """Create detailed text report"""
    report_path = output_dir / "detailed_analysis_report_ba.txt"

    with open(report_path, "w") as f:
        f.write("=" * 80 + "\n")
        f.write("🔬 BARABÁSI-ALBERT: MULTI-RANGE PAGERANK ANALYSIS REPORT\n")
        f.write("=" * 80 + "\n\n")

        all_deltas = [r["overall_mean_delta_pct"] for r in all_results]
        f.write("OVERALL STATISTICS\n")
        f.write("-" * 80 + "\n")
        f.write(f"Total Runs: {len(all_results)}\n")
        f.write(f"Overall Mean Delta: {np.mean(all_deltas):.4f}%\n")
        f.write(f"Overall Std Dev: {np.std(all_deltas):.4f}%\n")
        f.write(f"Range: [{np.min(all_deltas):.4f}%, {np.max(all_deltas):.4f}%]\n\n")

        best = max(all_results, key=lambda x: x["overall_mean_delta_pct"])
        f.write("BEST OVERALL COMBINATION\n")
        f.write("-" * 80 + "\n")
        f.write(f"Strategy: {best['strategy_name']}\n")
        f.write(
            f"Range: {best['range_name']} ({best['min_connections']}-{best['max_connections']})\n"
        )
        f.write(
            f"Mean Delta: {best['overall_mean_delta_pct']:.4f}% ± {best['confidence_interval_95']:.4f}%\n"
        )
        f.write(f"Efficiency: {best['efficiency']:.5f}% per connection\n")
        f.write(f"Total Simulations: {best['total_simulations']:,}\n\n")

        ranges = sorted(list(set([r["range_name"] for r in all_results])))
        f.write("ANALYSIS BY CONNECTION RANGE\n")
        f.write("=" * 80 + "\n\n")

        for range_name in ranges:
            f.write(f"{range_name}\n")
            f.write("-" * 80 + "\n")
            range_results = [r for r in all_results if r["range_name"] == range_name]
            range_results = sorted(
                range_results, key=lambda x: x["overall_mean_delta_pct"], reverse=True
            )

            for i, r in enumerate(range_results, 1):
                f.write(
                    f"  {i}. {r['strategy_name']:<25} "
                    f"{r['overall_mean_delta_pct']:>8.4f}% ± {r['confidence_interval_95']:>6.4f}%  "
                    f"Eff: {r['efficiency']:>7.5f}  "
                    f"↑{r['avg_pages_up']:>5.1f} ↓{r['avg_pages_down']:>5.1f}\n"
                )
            f.write("\n")

    print_progress(f"Saved: {report_path.name}", "success")


# ============================================
# MAIN EXECUTION
# ============================================

if __name__ == "__main__":
    print_progress("MULTI-RANGE BARABÁSI-ALBERT MODEL", "header")
    print(f"NetworKit version: {nk.__version__}")
    print()

    print_progress("Configuration Summary:", "info")
    print(f"  BA Graph: {TOTAL_NODES_WWW:,} nodes, m={EDGES_PER_NEW_NODE}")
    print(f"  PageRank: damping={PAGERANK_DAMPING}, tolerance={PAGERANK_TOLERANCE}")
    print(f"  Connection Ranges: {len(CONNECTION_RANGES)}")
    for min_c, max_c, name in CONNECTION_RANGES:
        print(f"    • {name}: {min_c}-{max_c} connections")
    print(f"  Strategies: {len(COMPARISON_FOLDERS)}")
    print(
        f"  Total runs: {len(CONNECTION_RANGES)} × {len(COMPARISON_FOLDERS)} = {len(CONNECTION_RANGES) * len(COMPARISON_FOLDERS)}"
    )
    print()

    # Mount drive
    mount_google_drive()

    # Initialize checkpoint manager
    checkpoint_mgr = CheckpointManager()
    existing_checkpoints = checkpoint_mgr.list_checkpoints()
    if existing_checkpoints:
        print_progress(
            f"Found {len(existing_checkpoints)} existing checkpoints", "info"
        )
        print("  (Will skip already completed runs)")
        print()

    # Verify baseline
    baseline_path = Path(BASELINE_PATH)
    if not baseline_path.exists():
        print_progress(f"Baseline not found: {BASELINE_PATH}", "error")
        exit(1)

    # Load baseline
    print_progress("Loading baseline graph...", "section")
    baseline_data = load_graph_from_csv_networkit(baseline_path)
    if baseline_data[0] is None:
        print_progress("Failed to load baseline", "error")
        exit(1)

    # Generate master BA graph
    master_www_graph = create_www_graph_networkit_OPTIMIZED(
        TOTAL_NODES_WWW, EDGES_PER_NEW_NODE, seed=42
    )

    # Process all combinations
    all_results = []
    total_runs = len(CONNECTION_RANGES) * len(COMPARISON_FOLDERS)
    current_run = 0
    failed_runs = []
    overall_start = time.time()

    for min_conn, max_conn, range_name in CONNECTION_RANGES:
        print(f"\n\n{'#' * 70}")
        print(f"# CONNECTION RANGE: {range_name} ({min_conn}-{max_conn})")
        print(f"{'#' * 70}")

        for folder_path, strategy_name in COMPARISON_FOLDERS:
            current_run += 1
            print(f"\n[RUN {current_run}/{total_runs}]")

            try:
                result = process_strategy_with_range(
                    baseline_data,
                    folder_path,
                    strategy_name,
                    master_www_graph,
                    min_conn,
                    max_conn,
                    range_name,
                    checkpoint_mgr,
                )

                if result is not None:
                    all_results.append(result)
                    print_progress(
                        f"{strategy_name} @ {range_name}: "
                        f"Mean Δ = {result['overall_mean_delta_pct']:+.4f}% "
                        f"± {result['confidence_interval_95']:.4f}%",
                        "success",
                    )
                else:
                    print_progress(
                        f"No results for {strategy_name} @ {range_name}", "error"
                    )
                    failed_runs.append((strategy_name, range_name))

            except Exception as e:
                print_progress(f"ERROR: {e}", "error")
                failed_runs.append((strategy_name, range_name))
                continue

    overall_elapsed = time.time() - overall_start

    # Save results
    if all_results:
        print_progress("SAVING RESULTS", "header")

        # CSV summary
        summary_df = pd.DataFrame(
            [
                {
                    "strategy": r["strategy_name"],
                    "range": r["range_name"],
                    "min_connections": r["min_connections"],
                    "max_connections": r["max_connections"],
                    "total_simulations": r["total_simulations"],
                    "mean_delta_pct": r["overall_mean_delta_pct"],
                    "ci_95": r["confidence_interval_95"],
                    "std_delta": r["overall_std"],
                    "avg_pages_up": r["avg_pages_up"],
                    "avg_pages_down": r["avg_pages_down"],
                    "efficiency": r["efficiency"],
                    "files_analyzed": r["files_analyzed"],
                }
                for r in all_results
            ]
        )

        csv_path = MULTI_RANGE_OUTPUT_DIR / "multi_range_results_ba.csv"
        summary_df.to_csv(csv_path, index=False)
        print_progress(f"Saved: {csv_path.name}", "success")

        # Create visualizations and report
        create_multi_range_comparison_plot(all_results, MULTI_RANGE_OUTPUT_DIR)
        create_detailed_analysis_report(all_results, MULTI_RANGE_OUTPUT_DIR)

        # Print summary
        print_progress("RESULTS SUMMARY", "header")
        print(
            f"Total strategies: {len(all_results)}/{len(COMPARISON_FOLDERS) * len(CONNECTION_RANGES)}"
        )
        print(
            f"Total runtime: {overall_elapsed / 3600:.2f} hours ({overall_elapsed / 60:.1f} minutes)"
        )
        print()

        for min_conn, max_conn, range_name in CONNECTION_RANGES:
            print(f"\n{range_name} ({min_conn}-{max_conn} connections):")
            print("-" * 70)
            range_results = [r for r in all_results if r["range_name"] == range_name]
            range_results = sorted(
                range_results, key=lambda x: x["overall_mean_delta_pct"], reverse=True
            )

            for i, r in enumerate(range_results, 1):
                print(
                    f"  {i}. {r['strategy_name']:<20} {r['overall_mean_delta_pct']:>8.4f}% ± {r['confidence_interval_95']:>6.4f}%  "
                    f"Eff: {r['efficiency']:>7.5f}"
                )

        # Key findings
        print(f"\n{'=' * 70}")
        print("KEY FINDINGS")
        print(f"{'=' * 70}")

        best_overall = max(all_results, key=lambda x: x["overall_mean_delta_pct"])
        print(f"\n🏆 BEST PERFORMANCE:")
        print(f"   Strategy: {best_overall['strategy_name']}")
        print(f"   Range: {best_overall['range_name']}")
        print(
            f"   Mean Delta: {best_overall['overall_mean_delta_pct']:+.4f}% ± {best_overall['confidence_interval_95']:.4f}%"
        )
        print(f"   Efficiency: {best_overall['efficiency']:.5f}% per connection")

        best_eff = max(all_results, key=lambda x: x["efficiency"])
        print(f"\n💰 BEST ROI:")
        print(f"   Strategy: {best_eff['strategy_name']}")
        print(f"   Range: {best_eff['range_name']}")
        print(f"   Efficiency: {best_eff['efficiency']:.5f}% per connection")

        if failed_runs:
            print(f"\n⚠️  FAILED RUNS: {len(failed_runs)}")

        print(f"\n{'=' * 70}")
        print("✓ ANALYSIS COMPLETE!")
        print(f"{'=' * 70}")
        print(f"\nOutput folder: {MULTI_RANGE_OUTPUT_DIR}")
        print("Files created:")
        print("  - multi_range_comparison_ba.png")
        print("  - detailed_analysis_report_ba.txt")
        print("  - multi_range_results_ba.csv")

    else:
        print_progress("No valid results obtained", "error")

    del master_www_graph
    gc.collect()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 48.5 MB/s eta 0:00:00


MULTI-RANGE BA MODEL CONFIGURATION
Simulations per run: 500 (20×25)
Connection Ranges:
  - Range_5-35: 5-35 connections
  - Range_35-65: 35-65 connections
  - Range_65-95: 65-95 connections

📁 Output: /content/drive/MyDrive/WebKnoGraph/results/multi_range_analysis_ba
📁 Checkpoints: /content/drive/MyDrive/WebKnoGraph/results/multi_range_analysis_ba/checkpoints_ba


[10:01:00] MULTI-RANGE BARABÁSI-ALBERT MODEL
NetworKit version: 11.1.post1

  • [10:01:00] Configuration Summary:
  BA Graph: 500,000 nodes, m=3
  PageRank: damping=0.8, tolerance=1e-06
  Connection Ranges: 3
    • Range_5-35: 5-35 connections
    • Range_35-65: 35-65 connections
    • Range_65-95: 65-95 connections
  Strategies: 5
  Total runs: 3 × 5 = 15

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  ✓ [10:01:01] Google Drive mounted successfully!
  • [10:01:01] Found 12 existing checkpoints
  (Will skip already completed runs)


──────────

/tmp/ipython-input-223187495.py:650: UserWarning: Glyph 128300 (\N{MICROSCOPE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-223187495.py:653: UserWarning: Glyph 128300 (\N{MICROSCOPE}) missing from font(s) DejaVu Sans.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


  ✓ [12:36:43] Saved: multi_range_comparison_ba.png
  ✓ [12:36:43] Saved: detailed_analysis_report_ba.txt

[12:36:43] RESULTS SUMMARY
Total strategies: 15/15
Total runtime: 2.59 hours (155.6 minutes)


Range_5-35 (5-35 connections):
----------------------------------------------------------------------
  1. Low Candidates         1.6276% ± 5.2093%  Eff: 0.08138
  2. Mixed Candidates       1.3672% ± 2.2791%  Eff: 0.06836
  3. High Candidates        1.3354% ± 5.0966%  Eff: 0.06677
  4. Random Candidates      1.2484% ± 1.4064%  Eff: 0.06242
  5. Folder Candidates      1.1530% ± 0.2362%  Eff: 0.05765

Range_35-65 (35-65 connections):
----------------------------------------------------------------------
  1. Low Candidates         2.1729% ± 5.3071%  Eff: 0.04346
  2. Mixed Candidates       1.9129% ± 2.2712%  Eff: 0.03826
  3. High Candidates        1.8765% ± 5.1311%  Eff: 0.03753
  4. Random Candidates      1.7920% ± 1.4294%  Eff: 0.03584
  5. Folder Candidates      1.6964% ± 0.2136%  Eff: